# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

In [8]:
import backtrader as bt
import datetime
import pandas as pd

## Define Strategies and Data utils

In [9]:
class BasicStrategy(bt.Strategy):
    def __init__(self):
        # To keep track of pending orders
        self.order = None
        print('***The limit sells for this strategy are valid for 1 day***')
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

            # Note: This logic does not properly work
#         elif order.status in [order.Expired]:
#             if order.isbuy():
#                 self.log('BUY ORDER EXPIRED')
#             elif order.issell():
#                 self.log('SELL ORDER EXPIRED')
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        self.order = None



    def next(self):
        # STRATEGY 
        data = self.datas[0]
        self.log(f'Current Portfolio Value : {self.broker.get_value()}')
        
        # Check if we are in the market
        if not self.position:
            # BUY
            try:
                self.size = int(self.broker.get_cash() / self.datas[0].open[1])
            except:
                print('Size Exception. If at the end of data, ignore.')
            # invest if prediction looks good
            # TODO: Make this a filter in the class constructor
            if (data.prediction[0] >= data.close[0]):
                self.log(f'MARKET BUY CREATE {self.size} shares at next open, current close price: {data.close[0]}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(size=self.size) # market order buys at next open
                
        else:
            # place sell order at predicted high if predicted high is greater than current close price
            if data.prediction[0] > data.close[0]:
                self.log(f'LIMIT SELL CREATE {self.size} shares at {data.prediction[0]}')
                # Keep track of the created order to avoid a 2nd order, order valid for 1 day
                self.order = self.sell(exectype=bt.Order.Limit,
                             price=data.prediction[0],
                             valid=data.datetime.date(0) + datetime.timedelta(days=2),
                             size=self.size)
            # if prediction is less than current value sell at open
            else:
                self.log('MARKET SELL CREATE. PREDICTION < CURRENT CLOSE')
                self.order = self.sell(size=self.size)
        

In [10]:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

      
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.datas[0].open[1])
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print(f'Stop price: {self.datas[0].close[0]}')
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [11]:
def prepare_data(data, fromdate, todate, filepath):
    """Prepare data for backtrader datafeed object
        Returns prepared data filepath and params for the GenericCSVData class, also returns columns used"""
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    # start param setup for backtrader
    start = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    from_to = [(start.year, start.month, start.day), (end.year, end.month, end.day)]
    # Backtrader string format
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # backtrader data feed class needs a file path, this could be a temp file that's constantly overwritten
    data.to_csv(filepath)
    
    starting_params = [
    ('fromdate', datetime.datetime(*from_to[0])),
    ('todate', datetime.datetime(*from_to[1])),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d %H:%M:%S')),
    ('tmformat', ('%H:%M:%S')),
    ('datetime', 1)]
    # skip nonfeatures (timestamp)
    cols = data.columns[1:]
    # get column position for each indicator and add to starting params list
    i = 2 # starting index since others are reserved
    for indicator in cols:
        starting_params.append((indicator, i))
        i+=1
    final_params = tuple(starting_params)

    return filepath, final_params, cols 


## Import Data and Run Backtest

In [12]:
import os

In [13]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../data/ticker_predictions/{pred_folder_name}'):
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df['prediction'] = pred_df[pred_col_name]
        # remove all non-prediction rows, use "timestamp" as the common datetime name
        ticker_df = ticker_df[ticker_df.prediction > 0]
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)

        # make timestamp in column position 0 
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']

        # skip nonfeatures, keep timestamp
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]

        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        ##### BUY AND HOLD #######
        print('BUY AND HOLD')
        # initialize everything and run strategy
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        #cerebro.addsizer(bt.sizers.FixedSize, stake=10)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BuyAndHold)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        buy_hold = cerebro.run()
        buy_hold_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(buy_hold[0].analyzers.sharpe.get_analysis())
        print(buy_hold[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(buy_hold[0].analyzers.returns.get_analysis())

        #### MAIN STRATEGY #####
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BasicStrategy)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        back = cerebro.run()
        strat_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(back[0].analyzers.sharpe.get_analysis())
        print(back[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(back[0].analyzers.returns.get_analysis())
        print(f"For {ticker_name}, return was {buy_hold_ret*100}% for buy and hold vs {strat_ret*100}%  for strategy")
        print('\n'*5)

        ticker_backtesting_dict[ticker_name] = cerebro
        
    return ticker_backtesting_dict
#     return ticker_df

In [14]:
# pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000

cerebro_dict = capstone_data_prep('arima_preds', 'prediction', '../data/ticker_data')
cerebro_dict

--------Ticker Name-------: WAT
BUY AND HOLD
Starting Portfolio Value: 100000
2000-04-03, BUY EXECUTED, 47.25
Stop price: 26.65999984741211
ROI:        -43.57%
Final Portfolio Value: 56431.55967712402

STRATEGY INFO:
OrderedDict([('sharperatio', -0.03987808414248128)])
AutoOrderedDict([('len', 416), ('drawdown', 70.49368783303537), ('moneydown', 134820.94032287598), ('max', AutoOrderedDict([('len', 416), ('drawdown', 74.93031505846456), ('moneydown', 143306.10080718994)]))])
OrderedDict([('rtot', -0.5721416152409546), ('ravg', -0.0010556118362379237), ('rnorm', -0.2335717426226032), ('rnorm100', -23.35717426226032)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2000-03-31, Current Portfolio Value : 100000.0
2000-03-31, MARKET BUY CREATE 2116 shares at next open, current close price: 47.625
2000-04-03, BUY EXECUTED, 47.25
2000-04-03, Current Portfolio Value : 92197.25
2000-04-03, LIMIT SELL CREATE 2116 shares at 230.130747057432
2000-04-04

2000-09-15, Current Portfolio Value : 161893.0
2000-09-15, LIMIT SELL CREATE 2116 shares at 210.07468795572493
2000-09-18, Current Portfolio Value : 161893.0
2000-09-18, LIMIT SELL CREATE 2116 shares at 200.48002328747543
2000-09-19, Current Portfolio Value : 157661.0
2000-09-19, LIMIT SELL CREATE 2116 shares at 201.56849500592213
2000-09-20, Current Portfolio Value : 159777.0
2000-09-20, LIMIT SELL CREATE 2116 shares at 192.08469362506412
2000-09-21, Current Portfolio Value : 169563.5
2000-09-21, LIMIT SELL CREATE 2116 shares at 198.26277291810143
2000-09-22, Current Portfolio Value : 170158.625
2000-09-22, LIMIT SELL CREATE 2116 shares at 185.5438058043336
2000-09-25, Current Portfolio Value : 183053.0
2000-09-25, LIMIT SELL CREATE 2116 shares at 168.3600885293876
2000-09-26, Current Portfolio Value : 171415.0
2000-09-26, LIMIT SELL CREATE 2116 shares at 185.92677548749492
2000-09-27, Current Portfolio Value : 173795.5
2000-09-27, LIMIT SELL CREATE 2116 shares at 175.2839614117352
20

2001-03-05, Current Portfolio Value : 133327.0
2001-03-05, LIMIT SELL CREATE 2116 shares at 216.74660021931516
2001-03-06, Current Portfolio Value : 130787.79838562012
2001-03-06, LIMIT SELL CREATE 2116 shares at 218.65142188198288
2001-03-07, Current Portfolio Value : 125455.4774169922
2001-03-07, LIMIT SELL CREATE 2116 shares at 216.41401035714065
2001-03-08, Current Portfolio Value : 126428.84355163574
2001-03-08, LIMIT SELL CREATE 2116 shares at 217.9358623366471
2001-03-09, Current Portfolio Value : 124228.20161437988
2001-03-09, LIMIT SELL CREATE 2116 shares at 216.70629328683447
2001-03-12, Current Portfolio Value : 104761.0
2001-03-12, LIMIT SELL CREATE 2116 shares at 217.83507193776185
2001-03-13, Current Portfolio Value : 107723.40322875977
2001-03-13, LIMIT SELL CREATE 2116 shares at 217.78467673831923
2001-03-14, Current Portfolio Value : 104761.0
2001-03-14, LIMIT SELL CREATE 2116 shares at 217.4520868761448
2001-03-15, Current Portfolio Value : 105184.20161437988
2001-03-

2001-08-27, Current Portfolio Value : 74988.88064575195
2001-08-27, LIMIT SELL CREATE 2116 shares at 285.0480736795837
2001-08-28, Current Portfolio Value : 74608.0
2001-08-28, LIMIT SELL CREATE 2116 shares at 286.17688308742254
2001-08-29, Current Portfolio Value : 73211.44032287598
2001-08-29, LIMIT SELL CREATE 2116 shares at 284.725587462827
2001-08-30, Current Portfolio Value : 72259.2387084961
2001-08-30, LIMIT SELL CREATE 2116 shares at 284.7961130608263
2001-08-31, Current Portfolio Value : 70122.08226013184
2001-08-31, LIMIT SELL CREATE 2116 shares at 283.00215468555007
2001-09-04, Current Portfolio Value : 71116.59677124023
2001-09-04, LIMIT SELL CREATE 2116 shares at 283.9293832956184
2001-09-05, Current Portfolio Value : 73021.0
2001-09-05, LIMIT SELL CREATE 2116 shares at 282.0044004774824
2001-09-06, Current Portfolio Value : 75158.15644836426
2001-09-06, LIMIT SELL CREATE 2116 shares at 281.06711435736366
2001-09-07, Current Portfolio Value : 73190.28387451172
2001-09-07,

2002-02-12, Current Portfolio Value : 75856.44032287598
2002-02-12, LIMIT SELL CREATE 2116 shares at 387.7170124466559
2002-02-13, Current Portfolio Value : 75771.79838562012
2002-02-13, LIMIT SELL CREATE 2116 shares at 385.3082541680175
2002-02-14, Current Portfolio Value : 75010.03709411621
2002-02-14, LIMIT SELL CREATE 2116 shares at 381.4885225757201
2002-02-15, Current Portfolio Value : 74184.79838562012
2002-02-15, LIMIT SELL CREATE 2116 shares at 389.2489219362127
2002-02-19, Current Portfolio Value : 73867.40322875977
2002-02-19, LIMIT SELL CREATE 2116 shares at 394.0865535135902
2002-02-20, Current Portfolio Value : 73021.0
2002-02-20, LIMIT SELL CREATE 2116 shares at 392.8670727307396
2002-02-21, Current Portfolio Value : 72280.40322875977
2002-02-21, LIMIT SELL CREATE 2116 shares at 391.8995525666464
2002-02-22, Current Portfolio Value : 71751.40322875977
2002-02-22, LIMIT SELL CREATE 2116 shares at 394.82228959285
2002-02-25, Current Portfolio Value : 69318.0
2002-02-25, LI

...

2000-10-23, Current Portfolio Value : 53025.53125
2000-10-23, LIMIT SELL CREATE 9922 shares at 149.96400449850336
2000-10-24, Current Portfolio Value : 52870.5
2000-10-24, LIMIT SELL CREATE 9922 shares at 156.8910252231213
2000-10-25, Current Portfolio Value : 53490.625
2000-10-25, LIMIT SELL CREATE 9922 shares at 143.62436489305006
2000-10-26, Current Portfolio Value : 54575.84375
2000-10-26, LIMIT SELL CREATE 9922 shares at 151.12862961825715
2000-10-27, Current Portfolio Value : 55971.125
2000-10-27, LIMIT SELL CREATE 9922 shares at 149.9032435987492
2000-10-30, Current Portfolio Value : 54885.90625
2000-10-30, LIMIT SELL CREATE 9922 shares at 145.97387391678393
2000-10-31, Current Portfolio Value : 54730.875
2000-10-31, LIMIT SELL CREATE 9922 shares at 151.86792843973183
2000-11-01, Current Portfolio Value : 52095.34375
2000-11-01, LIMIT SELL CREATE 9922 shares at 151.89830888960893
2000-11-02, Current Portfolio Value : 52560.4375
2000-11-02, LIMIT SELL CREATE 9922 shares at 162.92

2001-04-26, Current Portfolio Value : 87343.24647903442
2001-04-26, LIMIT SELL CREATE 9922 shares at 238.87093848885817
2001-04-27, Current Portfolio Value : 88211.42715644836
2001-04-27, LIMIT SELL CREATE 9922 shares at 233.6959240031587
2001-04-30, Current Portfolio Value : 86475.07526397705
2001-04-30, LIMIT SELL CREATE 9922 shares at 229.8577111237493
2001-05-01, Current Portfolio Value : 89451.67715644836
2001-05-01, LIMIT SELL CREATE 9922 shares at 227.99430475097336
2001-05-02, Current Portfolio Value : 90319.84837150574
2001-05-02, LIMIT SELL CREATE 9922 shares at 230.5362335925452
2001-05-03, Current Portfolio Value : 89823.74647903442
2001-05-03, LIMIT SELL CREATE 9922 shares at 232.48065964930151
2001-05-04, Current Portfolio Value : 90543.09375
2001-05-04, LIMIT SELL CREATE 9922 shares at 224.69281830370065
2001-05-07, Current Portfolio Value : 89575.70026397705
2001-05-07, LIMIT SELL CREATE 9922 shares at 223.7712522358881
2001-05-08, Current Portfolio Value : 89873.357614

2001-08-20, Current Portfolio Value : 91882.56602096558
2001-08-20, LIMIT SELL CREATE 9922 shares at 323.919416268502
2001-08-21, Current Portfolio Value : 91063.99647903442
2001-08-21, LIMIT SELL CREATE 9922 shares at 321.8534529593126
2001-08-22, Current Portfolio Value : 93172.42715644836
2001-08-22, LIMIT SELL CREATE 9922 shares at 324.2029774359712
2001-08-23, Current Portfolio Value : 94164.62147903442
2001-08-23, LIMIT SELL CREATE 9922 shares at 318.3291662443249
2001-08-24, Current Portfolio Value : 95752.13996505736
2001-08-24, LIMIT SELL CREATE 9922 shares at 315.06819736550466
2001-08-27, Current Portfolio Value : 95429.68177795409
2001-08-27, LIMIT SELL CREATE 9922 shares at 312.4351426272263
2001-08-28, Current Portfolio Value : 95032.80215644836
2001-08-28, LIMIT SELL CREATE 9922 shares at 314.92643223469474
2001-08-29, Current Portfolio Value : 96322.66329193115
2001-08-29, LIMIT SELL CREATE 9922 shares at 312.5060097397066
2001-08-30, Current Portfolio Value : 93693.329

2002-02-15, Current Portfolio Value : 103838.57526397705
2002-02-15, LIMIT SELL CREATE 9922 shares at 297.43671938953827
2002-02-19, Current Portfolio Value : 100465.09375
2002-02-19, LIMIT SELL CREATE 9922 shares at 301.5280975336151
2002-02-20, Current Portfolio Value : 98951.98723602295
2002-02-20, LIMIT SELL CREATE 9922 shares at 299.3608866620684
2002-02-21, Current Portfolio Value : 95231.23723602295
2002-02-21, LIMIT SELL CREATE 9922 shares at 299.8773543099787
2002-02-22, Current Portfolio Value : 102424.69102096558
2002-02-22, LIMIT SELL CREATE 9922 shares at 298.2165048736162
2002-02-25, Current Portfolio Value : 106864.78829193115
2002-02-25, LIMIT SELL CREATE 9922 shares at 299.1380863943533
2002-02-26, Current Portfolio Value : 107708.14920806885
2002-02-26, LIMIT SELL CREATE 9922 shares at 295.6441801292426
2002-02-27, Current Portfolio Value : 111478.51034355164
2002-02-27, LIMIT SELL CREATE 9922 shares at 300.1913268331743
2002-02-28, Current Portfolio Value : 109568.52

{'WAT': <backtrader.cerebro.Cerebro at 0x7f83d85286d0>,
 'UNH': <backtrader.cerebro.Cerebro at 0x7f83e9bcf310>,
 'HD': <backtrader.cerebro.Cerebro at 0x7f83e90ba5d0>,
 'MSFT': <backtrader.cerebro.Cerebro at 0x7f83ea26cbd0>,
 'XOM': <backtrader.cerebro.Cerebro at 0x7f83ea307690>,
 'ADSK': <backtrader.cerebro.Cerebro at 0x7f84097ce910>}

### Library additionally has plotting
Not sure how to reformat this, it's pretty ugly

In [22]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12}) 
cerebro_dict['UNH'].plot()

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/opt/anaconda3/lib/python3.7/site-packages/matplotlib/dates.py)

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open. Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements.  